# <center> <img src="../labs/img/ITESOLogo.png" alt="ITESO" width="480" height="130"> </center>
# <center> **Departamento de Electrónica, Sistemas e Informática** </center>
---
## <center> **Procesamiento de Datos Masivos** </center>
---
### <center> **Primavera 2025** </center>
---
### <center> **Lab 06: solution** </center>

---
**Profesor**: Dr. Pablo Camarillo Ramirez

In [ ]:
import findspark
findspark.init()

#### Creacion de la conexión con el cluster de spark


In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("SparkSQL-Lab06-Solution") \
    .master("spark://078b2e28e517:7077") \
    .config("spark.ui.port","4040") \
    .getOrCreate()
sc = spark.sparkContext

### Loading Netflix dataset

In [ ]:
from team_name.spark_utils import SparkUtils

columns_info = [("show_id", "string"),("type", "string"),("title", "string"),("director", "string"),("country", "string"),("date_added", "string"),("release_year", "integer"),("rating", "string"), ("duration", "string"),("listed_in", "string")]

netflix_schema = SparkUtils.generate_schema(columns_info)

netflix_df = spark.read \
                .schema(netflix_schema) \
                .option("header", "true") \
                .csv("/home/jovyan/notebooks/data/netflix1.csv")

netflix_df.printSchema()

netflix_df.show(5, truncate=False)

In [ ]:
from pyspark.sql.functions import count, when, col
df_nulls = netflix_df.select([count(when(col(c).isNull(), c)).alias(c) for c in netflix_df.columns])

df_nulls.collect()

#### Clean data

In [ ]:
clean_dict = {
    "director": "Uknown director",
    "country": "Uknown",
    "date_added": "Not provided",
    "release_year": 0,
    "rating": "Uknown",
    "duration": "Uknown",
    "listed_in": "Not provided"
}
netflix_df = SparkUtils.clean_df(netflix_df, clean_dict)

df_nulls = netflix_df.select([count(when(col(c).isNull(), c)).alias(c) for c in netflix_df.columns])

df_nulls.collect()


In [ ]:
write_opts_dict = {
    "mode": "overwrite",
    "parition_cols": ["release_year"],
    "format": "csv",
    "path": "/home/jovyan/notebooks/data/netflix_output"

}

SparkUtils.write_df(netflix_df, write_opts_dict)

In [ ]:
# Stop the SparkContext
sc.stop()